# 1. Import Library
pada tahapan ini Anda mendefinisikan terlebih dahulu library yang akan Anda gunakan 

In [1]:
from keras.datasets import mnist

In [2]:
from tensorflow.keras import Sequential
from keras.layers import BatchNormalization, Dense, Reshape, Flatten
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np

# 2. Mendefinisikan Variabel untuk Neural Network dan Data

#### Pada tahapan ini kita akan mendefinisikan beberapa variabel yang diperlukan  diantaranya

* Ukuran gambar yang akan di generate oleh algoritma GAN
* Channel warna yang akan digunakan
* Optimizer - Variable yang menentukan algoritma optimisasi pembelajaran yang akan dilakukan. Disini kita menggunakan algoritma Stochastic Gradient Decent dengan learning rate 0.0001. Learning rate mendefinisikan seberapa cepat algoritma mempelajari data yang diberikan, dan nilai 0.0001 adalah nilai yang direkomendasikan digunakan untuk pembentukan gan

In [3]:
## mendefinisikan variable gambar
## ukuran disesai
img_width = 28
img_height = 28
channels = 1
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(learning_rate=0.0001)

# 3. Membentuk Generator
Generator adalah bagian dari GAN yang bertugas untuk belajar membuat data palsu dengan memasukkan umpan balik dari diskriminator. Ia belajar membuat diskriminator mengklasifikasikan outputnya sebagai yang sebenarnya atau nyata.

In [4]:
def build_generator():
  model = Sequential()

  model.add(Dense(256, input_dim=latent_dim))
  ## add activation fucntion
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(512))
  ## add activation fucntion
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(1024))
  ## add activation fucntion
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  ## membuat model menjadi ukuran 28x28x1
  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))

  model.summary()
  return model

In [5]:
generator = build_generator()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               25856     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256)               0         
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                        

# 4. Mendefinisikan Discriminator
Diskriminator dalam GAN hanyalah sebuah fungsi klasifikasi. Ia mencoba untuk membedakan data nyata dari data yang dibuat oleh generator. Diskriminator bisa menggunakan arsitektur jaringan apa pun yang sesuai dengan jenis data yang diklasifikasikan.

In [6]:
def build_discriminator():
  model = Sequential()

  model.add(Flatten(input_shape=img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))

  model.summary()
  return model

discriminator = build_discriminator()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               401920    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 256)               0         
                                                                 
Total params: 533,248
Trainable params: 533,248
Non-trainable params: 0
_________________________________________________________________


# 5. Menghubungkan Discriminator dan Generator untuk membentuk GAN
Pada tahapan ini kita akan menghubungkan discriminator dan generator yang telah dibuat untuk membentuk sebuah GAN

In [7]:
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer='adam')